In [1]:
param_grid = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'colsample_bytree': [0.3, 0.7]
}
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1)
grid_search.fit(X_train, y_train)

# Step 9: Best model from Grid Search
best_model = grid_search.best_estimator_

# Step 10: Make Predictions
y_pred = best_model.predict(X_test)

# Step 11: Evaluate the Model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')
print('Classification Report:')
print(classification_report(y_test, y_pred))

# Step 12: Feature Importance (for model interpretability)
xgb.plot_importance(best_model)


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/match_score_dataset_basketball.csv'

## Dependencies

In [8]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score

##  Load the dataset
### Update file path accordingly

In [10]:
df = pd.read_csv('C:\\Users\\Lenovo\\Desktop\\match_score_dataset_basketball.csv')

## Data Exploration
### show basic info and check for missing values

In [11]:
print(df.info())  # Check column types and missing data
print(df.describe())  # Statistical summary

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710 entries, 0 to 709
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Edition Year  710 non-null    int64 
 1   Team 1        710 non-null    object
 2   Team 2        710 non-null    object
 3   Score 1       710 non-null    int64 
 4   Score 2       710 non-null    int64 
 5   Winner        710 non-null    object
dtypes: int64(3), object(3)
memory usage: 33.4+ KB
None
       Edition Year     Score 1     Score 2
count    710.000000  710.000000  710.000000
mean    1989.577465   79.695775   77.823944
std       16.764243   18.296349   17.741960
min     1964.000000    2.000000    0.000000
25%     1976.000000   68.000000   66.000000
50%     1988.000000   79.000000   77.000000
75%     2004.000000   91.000000   89.000000
max     2020.000000  156.000000  129.000000


## Data Preprocessing
### Handle missing values 

In [12]:
# Handle missing values for numeric columns only
numeric_columns = df.select_dtypes(include=[np.number]).columns
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())

# Convert categorical columns to numerical (e.g., team names)
label_encoder = LabelEncoder()
df['Team 1'] = label_encoder.fit_transform(df['Team 1'])
df['Team 2'] = label_encoder.fit_transform(df['Team 2'])

# Create a new target column: win/loss for team_1 (binary classification)
df['match_result'] = np.where(df['Score 1'] > df['Score 2'], 1, 0)

# Drop unnecessary columns (like scores if predicting only win/loss)
df.drop(['Score 1', 'Score 2'], axis=1, inplace=True)

## Feature Engineering


In [13]:
# Ensure 'location' column exists
if 'location' not in df.columns:
	df['location'] = 'home'  # or set to a default value like 'away'

# Create cumulative win columns for each team
df['team_1_wins'] = df.groupby('Team 1')['match_result'].cumsum()
df['team_2_wins'] = df.groupby('Team 2')['match_result'].transform(lambda x: x.cumsum().shift().fillna(0))

# Calculate recent wins ratio and home advantage
df['team_1_recent_wins'] = df['team_1_wins'] / (df['team_1_wins'] + df['team_2_wins'])
df['home_advantage'] = np.where(df['location'] == 'home', 1, 0)

## Define Features and Target

In [14]:

X = df[['Team 1', 'Team 2', 'team_1_recent_wins', 'home_advantage']]
y = df['match_result']

## Train-Test Split


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Initialize XGBoost Model


In [16]:
xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')

## Hyperparameter Tuning


In [3]:
param_grid = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'colsample_bytree': [0.3, 0.7]
}
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1)
grid_search.fit(X_train, y_train)

NameError: name 'GridSearchCV' is not defined